In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import metrics
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearnex import patch_sklearn
patch_sklearn()

In [2]:
data_2 = pd.read_csv(r'diabetes.csv')
data_2

data_2=data_2.fillna(0) # fill NAs  with 0 

X = data_2.drop(['Outcome'], axis=1)
y = data_2.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [3]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
# Select an algorithm
algorithm = KNeighborsClassifier()
seed = 13
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)
# Define our candidate hyperparameters
hp_candidates = [{'n_neighbors': [2,3,4,5,6], 'weights': ['uniform','distance']}]
# Search for best hyperparameters
grid = GridSearchCV(estimator=algorithm, param_grid=hp_candidates, cv=kfold, scoring='r2')
knn =grid.fit(X_train, y_train)

# Get the results
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_) 
# Create 3 folds

knn_predictions = knn.predict(X_test) 
cm = confusion_matrix(y_test, knn_predictions)
print(cm)
classification_report(y_test, knn_predictions)
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
print(accuracy)

-0.19942909038047643
KNeighborsClassifier(n_neighbors=4)
{'n_neighbors': 4, 'weights': 'uniform'}
[[146  21]
 [ 45  42]]
-0.15383027049349574


In [4]:
#Naïve Bayes
from sklearn.naive_bayes import GaussianNB
gnb_clf=GaussianNB()
parameters = {
    'var_smoothing': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15]
}
clf = GridSearchCV(gnb_clf, parameters, cv=5)
gnb=clf.fit(X_train, y_train)


gnb_predictions = gnb.predict(X_test)
  
# accuracy on X_test
accuracy = gnb.score(X_test, y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(y_test, gnb_predictions)
print(cm)
cls_report=classification_report(y_test, gnb_predictions)
print(cls_report)

0.7322834645669292
[[131  36]
 [ 32  55]]
              precision    recall  f1-score   support

           0       0.80      0.78      0.79       167
           1       0.60      0.63      0.62        87

    accuracy                           0.73       254
   macro avg       0.70      0.71      0.71       254
weighted avg       0.74      0.73      0.73       254



In [12]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
model = LogisticRegression(solver='liblinear')

c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=1, cv=cv)
grid_result = grid_search.fit(X_train, y_train)


logreg=model.fit(X_train, y_train)
log_predictions = model.predict(X_test)
print(log_predictions)
# accuracy on X_test
accuracy = grid_result.score(X_test, y_test)
print(accuracy)
  
# creating a confusion matrix
cm = confusion_matrix(y_test, log_predictions)
print(cm)
cl_repot=classification_report(y_test, log_predictions)
print(cl_repot)

[0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0 1
 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
0.7755905511811023
[[150  14]
 [ 41  49]]
              precision    recall  f1-score   support

           0       0.79      0.91      0.85       164
           1       0.78      0.54      0.64        90

    accuracy                           0.78       254
   macro avg       0.78      0.73      0.74       254
weighted avg       0.78      0.78      0.77       254



In [13]:
import pickle
with open('model_diabits','wb') as file:
    pickle.dump(gnb,file)

In [14]:
with open('model_diabits','rb') as file:
    mp = pickle.load(file)
    
pre=mp.predict(X_test)
print(pre) 

[0 0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 0 0 1
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1
 0 0 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 0 0 1 0]


In [5]:
#Decision Tree algorithms
from sklearn.tree import DecisionTreeClassifier
model= DecisionTreeClassifier()
parameters={
            "max_depth" : [7,9,11,12,15],
           "min_samples_leaf":[50,10,100],
           "min_weight_fraction_leaf":[0.0001,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
           "max_leaf_nodes":[None,1,20,30,40,50,60,70,80,90] }

tuning_model=GridSearchCV(estimator=model,param_grid=parameters,cv=3,verbose=3)

dtree_model =tuning_model.fit(X_train, y_train)

#dtree_model = DecisionTreeClassifier(max_depth = 15).fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)
accuracy = dtree_model.score(X_test, y_test)
print(accuracy)  
# creating a confusion matrix
cm = confusion_matrix(y_test, dtree_predictions)
print(cm)

cl_report=classification_report(y_test, dtree_predictions)
print(cl_report)

0.785486
[[141  26]
 [ 41  46]]
              precision    recall  f1-score   support

           0       0.77      0.84      0.81       167
           1       0.64      0.53      0.58        87

    accuracy                           0.74       254
   macro avg       0.71      0.69      0.69       254
weighted avg       0.73      0.74      0.73       254



In [1]:
import pickle
import numpy as np 
import pandas as pd
with open('model_diabits','rb') as file:
    mp = pickle.load(file)
    
    

In [7]:
X_test_new = [0,1,1,1,1,1,1,1] 
print(X_test_new)
datanew=pd.DataFrame(X_test_new).transpose()
pre=mp.predict(datanew)
print(pre)

[0, 1, 1, 1, 1, 1, 1, 1]
[0]


/home/dilum/.local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


[70, 1, 4, 130, 322, 0, 2, 109, 0, 2.4, 2, 3, 3, 130, 322, 0, 2, 109, 0, 2.4, 2, 3, 3]


/home/dilum/.local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


ValueError: X has 23 features, but GaussianNB is expecting 8 features as input.